In [1]:
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.layers import Dense,LSTM,Flatten,Dropout,Input,concatenate
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,r2_score
import tensorflow as tf
from sklearn.model_selection import train_test_split

# my_seed = 7
# tf.compat.v1.set_random_seed(my_seed)
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
    assert tf.config.experimental.get_memory_growth(physical_devices[0])
except:
    pass

In [2]:
help(Dense)

Help on class Dense in module keras.layers.core.dense:

class Dense(keras.engine.base_layer.Layer)
 |  Dense(*args, **kwargs)
 |  
 |  Just your regular densely-connected NN layer.
 |  
 |  `Dense` implements the operation:
 |  `output = activation(dot(input, kernel) + bias)`
 |  where `activation` is the element-wise activation function
 |  passed as the `activation` argument, `kernel` is a weights matrix
 |  created by the layer, and `bias` is a bias vector created by the layer
 |  (only applicable if `use_bias` is `True`). These are all attributes of
 |  `Dense`.
 |  
 |  Note: If the input to the layer has a rank greater than 2, then `Dense`
 |  computes the dot product between the `inputs` and the `kernel` along the
 |  last axis of the `inputs` and axis 0 of the `kernel` (using `tf.tensordot`).
 |  For example, if input has dimensions `(batch_size, d0, d1)`,
 |  then we create a `kernel` with shape `(d1, units)`, and the `kernel` operates
 |  along axis 2 of the `input`, on every

In [3]:
data = pd.read_csv('data/da1.csv',index_col='time')
data = data.iloc[:,[0,1,2,5,6,7]]
data

,number,weekday,isRest,severity_num,priority_num,category
time,,,,,,
2002/9/23,206,2,0,18,13,3
2002/9/24,192,3,0,29,18,3
2002/9/25,245,4,0,34,16,3
2002/9/26,209,5,0,18,20,3
2002/9/27,225,6,0,30,27,3
...,...,...,...,...,...,...
2004/5/10,108,2,0,11,6,2
2004/5/11,110,3,0,18,8,2
2004/5/12,114,4,0,15,8,2


In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(data.iloc[:,[0]].values)
# dataset = scaler.fit_transform(data.iloc[:,[0,1,2,3,4]].values)

In [5]:
dataset

array([[0.67672414],
       [0.61637931],
       [0.84482759],
       [0.68965517],
       [0.75862069],
       [0.45258621],
       [0.43103448],
       [0.75      ],
       [0.72844828],
       [0.70689655],
       [0.84051724],
       [0.64655172],
       [0.25      ],
       [0.47844828],
       [0.68965517],
       [0.71982759],
       [0.82327586],
       [0.76724138],
       [0.60344828],
       [0.24137931],
       [0.2887931 ],
       [0.65086207],
       [0.63362069],
       [0.71982759],
       [1.        ],
       [0.79741379],
       [0.44396552],
       [0.34913793],
       [0.7887931 ],
       [0.77586207],
       [0.86206897],
       [0.75431034],
       [0.64224138],
       [0.31465517],
       [0.21982759],
       [0.52586207],
       [0.61206897],
       [0.73706897],
       [0.68534483],
       [0.48706897],
       [0.27155172],
       [0.2887931 ],
       [0.41810345],
       [0.67672414],
       [0.68965517],
       [0.72844828],
       [0.38793103],
       [0.495

In [6]:
def create_dataset(data,dataset,lookback):
    dataX,dataY = [],[]
    for i in range(len(data) - lookback):
        a = dataset[i:i+lookback,[0,1,2,3,4]]
#         a = dataset[i:i+lookback,[0,1,2]]
#         print(a)
        dataX.append(a)
        dataY.append(data.iloc[i+lookback,[5]])
    return np.array(dataX),np.array(dataY)

In [7]:
def create_dataset(data,dataset,lookback):
    dataX,dataY = [],[]
    for i in range(len(data) - lookback):
#         a = dataset[i:i+lookback,[0,1,2,3,4]]
        a = dataset[i:i+lookback,[0]]
#         print(a)
        dataX.append(a)
        dataY.append(data.iloc[i+lookback,[5]])
    return np.array(dataX),np.array(dataY)

In [8]:
# dataX,dataY = create_dataset(data,dataset,7)
look_back = 7
dataX,dataY = create_dataset(data,dataset,look_back)

In [9]:
dataX.shape,dataY.shape

((593, 7, 1), (593, 1))

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
# 将category列进行one-hot编码
one_hots = to_categorical(dataY-1)

In [12]:
one_hots.shape

(593, 3)

In [13]:
trainX,testX,trainY,testY = train_test_split(dataX,one_hots,test_size=0.20, random_state=1)
# trainX,testX,trainY,testY = train_test_split(dataX,one_hots,test_size=0.20)

In [14]:
trainX.shape,testX.shape,trainY.shape,testY.shape

((474, 7, 1), (119, 7, 1), (474, 3), (119, 3))

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [16]:
model = Sequential()
model.add(LSTM(16, input_shape=(trainX.shape[1],trainX.shape[2])))
# model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(trainX, trainY, epochs=100, batch_size=12, verbose=2,validation_split=0.2)
model.fit(trainX, trainY, epochs=100, batch_size=128, verbose=2,validation_split=0.2,callbacks = [callback])

Epoch 1/100
3/3 - 1s - loss: 1.0995 - accuracy: 0.3325 - val_loss: 1.0992 - val_accuracy: 0.3684 - 1s/epoch - 417ms/step
Epoch 2/100
3/3 - 0s - loss: 1.0984 - accuracy: 0.3826 - val_loss: 1.0980 - val_accuracy: 0.4316 - 22ms/epoch - 7ms/step
Epoch 3/100
3/3 - 0s - loss: 1.0972 - accuracy: 0.4063 - val_loss: 1.0969 - val_accuracy: 0.4105 - 20ms/epoch - 7ms/step
Epoch 4/100
3/3 - 0s - loss: 1.0960 - accuracy: 0.4090 - val_loss: 1.0957 - val_accuracy: 0.3474 - 21ms/epoch - 7ms/step
Epoch 5/100
3/3 - 0s - loss: 1.0948 - accuracy: 0.4090 - val_loss: 1.0944 - val_accuracy: 0.4000 - 24ms/epoch - 8ms/step
Epoch 6/100
3/3 - 0s - loss: 1.0937 - accuracy: 0.3984 - val_loss: 1.0931 - val_accuracy: 0.3579 - 24ms/epoch - 8ms/step
Epoch 7/100
3/3 - 0s - loss: 1.0922 - accuracy: 0.3720 - val_loss: 1.0917 - val_accuracy: 0.3474 - 19ms/epoch - 6ms/step
Epoch 8/100
3/3 - 0s - loss: 1.0911 - accuracy: 0.3668 - val_loss: 1.0903 - val_accuracy: 0.3368 - 20ms/epoch - 7ms/step
Epoch 9/100
3/3 - 0s - loss: 1.0

Epoch 69/100
3/3 - 0s - loss: 0.8650 - accuracy: 0.5752 - val_loss: 0.7998 - val_accuracy: 0.6421 - 17ms/epoch - 6ms/step
Epoch 70/100
3/3 - 0s - loss: 0.8652 - accuracy: 0.5778 - val_loss: 0.8039 - val_accuracy: 0.6421 - 18ms/epoch - 6ms/step
Epoch 71/100
3/3 - 0s - loss: 0.8632 - accuracy: 0.5831 - val_loss: 0.8049 - val_accuracy: 0.6421 - 17ms/epoch - 6ms/step
Epoch 72/100
3/3 - 0s - loss: 0.8634 - accuracy: 0.5884 - val_loss: 0.8029 - val_accuracy: 0.6421 - 18ms/epoch - 6ms/step
Epoch 73/100
3/3 - 0s - loss: 0.8621 - accuracy: 0.5831 - val_loss: 0.8042 - val_accuracy: 0.6421 - 17ms/epoch - 6ms/step
Epoch 74/100
3/3 - 0s - loss: 0.8622 - accuracy: 0.5884 - val_loss: 0.8075 - val_accuracy: 0.6526 - 17ms/epoch - 6ms/step
Epoch 75/100
3/3 - 0s - loss: 0.8636 - accuracy: 0.5831 - val_loss: 0.8027 - val_accuracy: 0.6632 - 17ms/epoch - 6ms/step
Epoch 76/100
3/3 - 0s - loss: 0.8604 - accuracy: 0.5858 - val_loss: 0.8005 - val_accuracy: 0.6526 - 18ms/epoch - 6ms/step
Epoch 77/100
3/3 - 0s - 

In [17]:
accuracy = model.evaluate(testX, testY, batch_size=128, verbose=2)

1/1 - 0s - loss: 0.8651 - accuracy: 0.5798 - 10ms/epoch - 10ms/step


In [18]:
testPredict = model.predict(testX)

In [19]:
# model = load_model('model/lstm11.h5')
# model = load_model('lstm11.h5')
# testPredict = model.predict(testX)

In [20]:
y_pred = np.argmax(testPredict, axis=1)

In [21]:
y_pred

array([1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0, 1, 0, 2, 2, 0, 2, 0, 1, 1, 2, 2,
       0, 2, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 1,
       0, 1, 2, 1, 0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 2, 0, 2, 0, 1,
       1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 1, 1, 1, 1, 0, 2, 0, 0, 1, 1,
       1, 1, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 1, 1, 2, 2, 2, 1,
       0, 2, 1, 1, 2, 2, 2, 0, 1], dtype=int64)

In [22]:
y_real = np.argmax(testY, axis=1)

In [23]:
y_real

array([0, 0, 1, 2, 0, 1, 1, 0, 1, 2, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2,
       0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 2, 2, 1, 0, 1, 2, 1, 2, 1,
       0, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 0, 1,
       0, 0, 1, 0, 0, 1, 2, 2, 1, 0, 0, 2, 0, 1, 2, 2, 1, 0, 0, 0, 2, 1,
       1, 0, 0, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 2, 2, 2, 0,
       1, 1, 1, 0, 2, 2, 0, 1, 1], dtype=int64)

In [24]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_real,y_pred)
# kappa = cohen_kappa_score(dataY[-30:],y_pred)
#(label除非是你想计算其中的分类子集的kappa系数，否则不需要设置)
kappa

0.36472346786248133

In [25]:
from sklearn.metrics import classification_report

In [26]:
print(classification_report(y_real,y_pred))
# print(classification_report(dataY[-30:],y_pred))

              precision    recall  f1-score   support

           0       0.59      0.64      0.62        45
           1       0.51      0.40      0.45        45
           2       0.63      0.76      0.69        29

    accuracy                           0.58       119
   macro avg       0.58      0.60      0.58       119
weighted avg       0.57      0.58      0.57       119



In [27]:
# model.save('lstm11.h5')

In [28]:
classification_report(y_real,y_pred,output_dict=True)

{'0': {'precision': 0.5918367346938775,
  'recall': 0.6444444444444445,
  'f1-score': 0.6170212765957447,
  'support': 45},
 '1': {'precision': 0.5142857142857142,
  'recall': 0.4,
  'f1-score': 0.45,
  'support': 45},
 '2': {'precision': 0.6285714285714286,
  'recall': 0.7586206896551724,
  'f1-score': 0.6875,
  'support': 29},
 'accuracy': 0.5798319327731093,
 'macro avg': {'precision': 0.5782312925170068,
  'recall': 0.6010217113665389,
  'f1-score': 0.584840425531915,
  'support': 119},
 'weighted avg': {'precision': 0.5714628708626307,
  'recall': 0.5798319327731093,
  'f1-score': 0.571037457536206,
  'support': 119}}